In [1]:
import sys

from data import crop_upper_part, BinaryDataset
from model import SqueezeModel
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn import metrics
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import os
from PIL import Image

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
ANNOTATED_TEST_DATASET = '/Users/filipgulan/college/test'
CPU_CORES = 4
BATCH_SIZE = 32
USE_GPU = False

In [3]:
# THIS REQUIRES THE master branch of the `torchivsion` package
def data_transformations(input_shape, crop_perc = 0.5):
    return transforms.Compose([
        transforms.Lambda(lambda x: crop_upper_part(np.array(x), crop_perc)),
        transforms.ToPILImage(),
        transforms.Grayscale(3),
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
    ])

def var(tensor, volatile=False):
    if USE_GPU:
        tensor = tensor.cuda(0)
    return Variable(tensor, volatile=volatile)

def test_sigmoid(model, test_loader):
    model.eval()
    predictions, ground_truths = [], []
    for i, test_batch in enumerate(test_loader):
        test_x, test_y = (var(test_batch[0], volatile=True),
                          var(test_batch[1], volatile=True))
        logit = model(test_x)
        
        softmax = F.sigmoid(logit).cpu().data.numpy()
        ground_truths.extend(test_y.cpu().data.numpy())
        
        for act in softmax:
            predictions.append(act)
       
        print('Predicting batch {}/{}'.format(i + 1, len(test_loader)), end="\r", flush=True)
        
    return np.array(predictions), np.array(ground_truths)


def test_softmax(model, test_loader):
    model.eval()
    predictions, ground_truths = [], []
    for i, test_batch in enumerate(test_loader):
        test_x, test_y = (var(test_batch[0], volatile=True),
                          var(test_batch[1], volatile=True))
        logit = model(test_x)
        
        softmax = F.softmax(logit, dim=0).cpu().data.numpy()
        ground_truths.extend(test_y.cpu().data.numpy())
        
        for act in softmax:
            predictions.append(act)
       
        print('Predicting batch {}/{}'.format(i + 1, len(test_loader)), end="\r", flush=True)
        
    return np.array(predictions), np.array(ground_truths)
    

In [4]:
# Load sigmoid model
NUM_CLASSES = 26
MODEL_PATH = "./outdir/model"

model_sigmoid = SqueezeModel(fine_tune=True, num_classes=NUM_CLASSES)

model_sigmoid.load_state_dict(torch.load(MODEL_PATH, map_location=lambda storage, loc: storage))
print("Loaded model from:", MODEL_PATH)
model_sigmoid.eval()

if USE_GPU:
    model_sigmoid.cuda(0)

Loaded model from: ./outdir/model


/Users/filipgulan/college/mozgalo-mb/src/pytorch/squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/Users/filipgulan/college/mozgalo-mb/src/pytorch/squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [5]:
# Load softmax model
NUM_CLASSES = 26
MODEL_PATH = "./outdir/model"

model_softmax = SqueezeModel(fine_tune=True, num_classes=NUM_CLASSES)

model_softmax.load_state_dict(torch.load(MODEL_PATH, map_location=lambda storage, loc: storage))
print("Loaded model from:", MODEL_PATH)
model_softmax.eval()

if USE_GPU:
    model_softmax.cuda(0)

Loaded model from: ./outdir/model


/Users/filipgulan/college/mozgalo-mb/src/pytorch/squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/Users/filipgulan/college/mozgalo-mb/src/pytorch/squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [6]:
test_transform = data_transformations((3, 299, 299))

In [10]:
test_dataset_binary = BinaryDataset(images_dir=ANNOTATED_TEST_DATASET, transform=test_transform)
test_dataset_loader_binary = torch.utils.data.DataLoader(test_dataset_binary,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=False,
                                                    num_workers=CPU_CORES)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/filipgulan/college/test/Other'

In [7]:
test_dataset_softmax = datasets.ImageFolder(root=ANNOTATED_TEST_DATASET, transform=test_transform)
test_dataset_loader_softmax = torch.utils.data.DataLoader(test_dataset_softmax,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=False,
                                                    num_workers=CPU_CORES)

## Other vs. All

In [13]:
pred, true = test_sigmoid(model_sigmoid, test_dataset_loader_binary)

In [28]:
thr = 0.5
sigmoid_classes = np.copy(pred)
sigmoid_classes[pred > thr] = 1
sigmoid_classes[pred <= thr] = 0

report = metrics.classification_report(true, sigmoid_classes)
print(report)

             precision    recall  f1-score   support

        0.0       0.95      0.99      0.97      5009
        1.0       0.99      0.95      0.97      4991

avg / total       0.97      0.97      0.97     10000



## 25 class softmax

In [15]:
pred, true = test_softmax(model_softmax, test_dataset_loader_softmax)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


TypeError: softmax(): argument 'input' (position 1) must be Tensor, not tuple

In [ ]:
correct = np.argmax(pred)
report = metrics.classification_report(correct, true)
print(report)

# Exporting results for upload

In [8]:
# Class indices when OTher is present
labels = {'Albertsons': 0,
 'BJs': 1,
 'CVSPharmacy': 2,
 'Costco': 3,
 'FredMeyer': 4,
 'Frys': 5,
 'HEB': 6,
 'HarrisTeeter': 7,
 'HyVee': 8,
 'JewelOsco': 9,
 'KingSoopers': 10,
 'Kroger': 11,
 'Meijer': 12,
 'Other': 13,
 'Publix': 14,
 'Safeway': 15,
 'SamsClub': 16,
 'ShopRite': 17,
 'Smiths': 18,
 'StopShop': 19,
 'Target': 20,
 'Walgreens': 21,
 'Walmart': 22,
 'Wegmans': 23,
 'WholeFoodsMarket': 24,
 'WinCoFoods': 25}
class_dict = {v: k for k, v in labels.items()}
class_dict

{0: 'Albertsons',
 1: 'BJs',
 2: 'CVSPharmacy',
 3: 'Costco',
 4: 'FredMeyer',
 5: 'Frys',
 6: 'HEB',
 7: 'HarrisTeeter',
 8: 'HyVee',
 9: 'JewelOsco',
 10: 'KingSoopers',
 11: 'Kroger',
 12: 'Meijer',
 13: 'Other',
 14: 'Publix',
 15: 'Safeway',
 16: 'SamsClub',
 17: 'ShopRite',
 18: 'Smiths',
 19: 'StopShop',
 20: 'Target',
 21: 'Walgreens',
 22: 'Walmart',
 23: 'Wegmans',
 24: 'WholeFoodsMarket',
 25: 'WinCoFoods'}

In [ ]:
torch.set_num_threads(4)
IMAGES_FOLDER = "/Users/filipgulan/college/test/Other"
CSV_PATH = "/tmp/results.csv"

# TODO Add batching
results = []
images = sorted([(x, int(x.split(".")[0])) for x in os.listdir(IMAGES_FOLDER)], key=lambda x: x[1])

for ind, (img, _) in enumerate(images):
    with open(os.path.join(IMAGES_FOLDER, img), 'rb') as f:
        image = Image.open(f).convert("RGB")
        
    # make example a torch tensor
    value = test_transform(image)

    test_value = value.unsqueeze(0)
    test_value = Variable(test_value)
    if USE_GPU:
        test_value = test_value.cuda(0)
    
    prediction = model_softmax(test_value)[0].max(1)[1].cpu().data.numpy()[0]
    # Get names dicitonary
    name = class_dict[prediction]
    results.append(name)

        
    if ind % 1000 == 0:
        print("Processed {}/{}".format(ind, len(images)))
            

Processed 0/10000


In [15]:
results[:10]

['Smiths',
 'Target',
 'Other',
 'Smiths',
 'ShopRite',
 'Walgreens',
 'HarrisTeeter',
 'Other',
 'Safeway',
 'Smiths']

In [17]:
# Save to csv
with open(CSV_PATH, "w") as f:
    for i, line in enumerate(results):
        f.write(line)
        if i < len(results) - 1:
            f.write("\n")